# **Der Avocado-Erkenner**

### 1.) Foto hochladen

In [ ]:
from google.colab import files
userfile = files.upload()

Saving IMG_20230322_125742408_HDR.jpg to IMG_20230322_125742408_HDR.jpg


### 2.) Das Kernstück: Das neurale Netzwerk erkennt die Obst-/ Gemüsesorte

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import PIL
import io

## Finetuning the covnet

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 36)  # hier Anzahl der Kategorien

#model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load("/content/drive/MyDrive/TechLabs/model_ft_36_classes.pt"))
model_ft.eval();

categories = ['apple',
 'banana',
 'beetroot',
 'bell pepper',
 'cabbage',
 'capsicum',
 'carrot',
 'cauliflower',
 'chilli pepper',
 'corn',
 'cucumber',
 'eggplant',
 'garlic',
 'ginger',
 'grapes',
 'jalepeno',
 'kiwi',
 'lemon',
 'lettuce',
 'mango',
 'onion',
 'orange',
 'paprika',
 'pear',
 'peas',
 'pineapple',
 'pomegranate',
 'potato',
 'raddish',
 'soy beans',
 'spinach',
 'sweetcorn',
 'sweetpotato',
 'tomato',
 'turnip',
 'watermelon']

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


fnlist = []
imglist = []
for fn, fdata in userfile.items():
    img = PIL.Image.open(io.BytesIO(fdata))
    imglist.append(data_transforms['val'](img))
    fnlist.append(fn)
imgbatch = torch.stack(imglist)

model_ft.eval()
with torch.inference_mode():
    probs, preds = model_ft(imgbatch).softmax(dim=1).max(1)
probs = probs.tolist()
preds = [categories[idx] for idx in preds.tolist()]

result = dict(zip(fnlist, preds))
fruit = list(result.values())[0]


#Ausgabe vom Klimascore über ein dictionary
co2dict = {"apple": "0.525",
    "banana": "0.757",
    "beetroot": "",
    "bell pepper": "3.710",
    "cabbage": "1.599",
    "capsicum": "3.710",
    "carrot": "0.400",
    "cauliflower": "1.362",
    "chilli pepper": "",
    "corn": "",
    "cucumber": "1.878",
    "eggplant": "",
    "garlic": "",
    "ginger": "",
    "grapes": "1.100",
    "jalepeno": "",
    "kiwi": "0.721",
    "lemon": "1.152",
    "lettuce": "0.686",
    "mango": "1.506",
    "onion": "0.438",
    "orange": "0.783",
    "paprika": "3.710",
    "pear": "",
    "peas": "1.137",
    "pineapple": "1.005",
    "pomegranate": "",
    "potato": "0.923",
    "raddish": "",
    "soy beans": "",
    "spinach": "",
    "sweetcorn": "",
    "sweetpotato": "",
    "tomato": "1.792",
    "turnip": "",
    "watermelon": ""
}
co2_emission = co2dict[fruit]

# Making three categories (low, average, high) based on all fruit and vegatable CO2 information itn the original database
def co2_score(co2_emission):
    if not co2_emission:
        return "no info"
    co2_emission = float(co2_emission)
    if co2_emission < 1.168:
        return "This is a low climate impact compared to other fruit and vegetable. Good choice!"
    elif co2_emission <= 2.379:
        return "This is an average climate impact compared to other fruit and vegetable."
    else:
        return "This is a high climate impact compared to other fruit and vegetable. Maybe you can find an alternative?"

co2_context = co2_score(co2_emission)

### 3.) Ausgabe der Obst-/ Gemüsesorte

In [ ]:
f'This picture shows a(n) {fruit}!'

'This picture shows a(n) apple!'

### 4.) Ausgabe des Klimascores

In [ ]:
f'The amount of CO2 emission of 1 kg {fruit} is {co2_emission} kg CO2! {co2_context}'

'The amount of CO2 emission of 1 kg apple is 0.525 kg CO2! This is a low climate impact compared to other fruit and vegetable. Good choice!'